pillcam dataset split into neg/pos and trained using resnet.py witouth augmentation or ds normalization.

### Loading images
https://www.tensorflow.org/tutorials/load_data/images

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler

from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50

AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
model_name = '10x32x224_binary_resnet50' # epoch*batch_size*img_size

NUM_EPOCHS = 10
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CHANNELS = 3

BASE_LEARNING_RATE = 0.1
LR_SCHEDULE = [(0.1, 10), (0.01, 20)]  # [(0.1, 30), (0.01, 45)]

In [ ]:
data_dir = pathlib.Path('/mnt/sdb/augere_export_class/')

DATASET_SIZE = len(list(data_dir.glob('[!Anatomic]*/*.png')))
class_names = np.array([item.name for item in data_dir.glob('*') if item.name != 'metadata.json'])
# Remove 'anatomic landmarks'
original_class_names = np.delete(class_names, 0)
class_names = np.array(['Negative', 'Positive'])

neg_class_name = 'Normal'
pos_class_names = np.delete(original_class_names, np.where(neg_class_name == original_class_names))

NUM_CLASSES = 2

print ("Positive class names: ", pos_class_names)
print ("Negative class names: ", neg_class_name)

In [ ]:
# Create a dataset of the file paths
list_ds = tf.data.Dataset.list_files(str(data_dir/'[!Anatomic]*/*'))

In [ ]:
# Extract some info about each class

negative_samples = 0
positive_samples = 0
for class_name in original_class_names:
    # Number of samples in 'class_name' folder
    class_samples = len(list(data_dir.glob(class_name+'/*.png')))
    
    if (class_name != 'Normal'):
        negative_samples += class_samples
    else:
        positive_samples += class_samples

print ('Negative samples: {0:5} | {1:5.2f}%'.format(negative_samples, negative_samples/DATASET_SIZE*100))
print ('Positive samples: {0:5} | {1:5.2f}%'.format(positive_samples, positive_samples/DATASET_SIZE*100))
print ('\nTotal number of images:', DATASET_SIZE)

Set up pipeline for loading images from given list of paths

In [ ]:
def get_label_int(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    bc = parts[-2] == pos_class_names
    nz_cnt = tf.math.count_nonzero(bc)
    if (nz_cnt > 0):
        return tf.reshape(tf.dtypes.cast(tf.fill([1, 1], value=1), tf.uint8),[-1])
    return tf.reshape(tf.dtypes.cast(tf.fill([1, 1], value=0), tf.uint8),[-1])

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(file_path):
    label = get_label_int(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

Showing an example image/label pair

In [ ]:
def show_image(img):
    for image, label in img:
        plt.figure()
        plt.figure(frameon=False, facecolor='white')
        plt.title(class_names[label.numpy()][0], fontdict={'color':'white','size':20})
        plt.imshow(image.numpy())
        plt.axis('off')

show_image(labeled_ds.take(1))

### Splitting into training, test and validation data

In [ ]:
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

shuffled_ds = labeled_ds#.shuffle(buffer_size=10000)

train_ds = shuffled_ds.take(train_size)
test_ds = shuffled_ds.skip(train_size)
val_ds = test_ds.skip(val_size)
test_ds = test_ds.take(test_size)

In [ ]:
# Print info about the dataset split
def get_size(ds):
    return tf.data.experimental.cardinality(ds).numpy()

print ("{:32} {:>5}".format("Full dataset sample size:", get_size(shuffled_ds)))
print ("{:32} {:>5}".format("Train dataset sample size:", get_size(train_ds)))
print ("{:32} {:>5}".format("Test dataset sample size:", get_size(test_ds)))
print ("{:32} {:>5}".format("Validation dataset sample size:", get_size(val_ds)))

In [ ]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=3000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
      if isinstance(cache, str):
        ds = ds.cache(cache)
      else:
        ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

# Create training, test and validation dataset
train_ds = prepare_for_training(train_ds, cache="./cache/"+model_name+"_train_ds.tfcache")
test_ds = prepare_for_training(test_ds, cache="./cache/"+model_name+"binary_test_ds.tfcache")
val_ds = prepare_for_training(val_ds, cache="./cache/"+model_name+"binary_val_ds.tfcache")

## Training
towards data science resnet50: https://towardsdatascience.com/deep-learning-using-transfer-learning-python-code-for-resnet50-8acdfb3a2d38

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

In [ ]:
input_shape = (IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

output = resnet.layers[-1].output
output = keras.layers.Flatten()(output)

resnet = Model(inputs=resnet.input, outputs=output)

for layer in resnet.layers:
    layer.trainable = False
    
resnet.summary()

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers

model = Sequential()
model.add(resnet)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
        train_ds,
        steps_per_epoch = train_size // BATCH_SIZE,
        epochs = NUM_EPOCHS,
        validation_data = test_ds,
        validation_steps = test_size // BATCH_SIZE,
        validation_freq = 1)

In [ ]:
model.save('models/'+model_name+'.h5')
# new_model = keras.models.load_model('model.h5')

## Validate the results

`Tensorboard`

In [ ]:
# Clear any logs from previous runs (move to .old instead?)
# !rm -rf ./logs/

In [ ]:
# Start tensorboard
%tensorboard --logdir logs

In [ ]:
# !kill 20058

`Keras`
Save/load the model

In [ ]:
result = model.evaluate(
            val_ds,
            steps = val_size//BATCH_SIZE)

print (result)

## Predict new samples

In [ ]:
# Predict 1 image
image, label = next(iter(train_ds))
image = image.numpy()[0]
label = label.numpy()[0]

res = model.predict(np.expand_dims(image, axis=0))

count = 0
for number in res[0]:
    print("{:0.7f} {}".format(number,class_names[count]))
    count += 1
print()

print ("True label:", class_names[label[0]])
print ("Predicted label:", class_names[np.argmax(res[0])])

plt.figure()
plt.imshow(image)
plt.axis('off');

In [ ]:
image, label = next(iter(train_ds))
image = image.numpy()
label = label.numpy()

results = model.predict(image)

idx = 0
for res in results:
    true_label = class_names[label[idx]][0]
    pred_label = class_names[np.argmax(res)]
    print ('{:3}  Label: {:20} Prediction: {}'.format(idx, true_label, pred_label))
    idx += 1